# Orte für die Utrechter Priester für WIAG 2

[Zuordnung zu Orten in `place`](#Zuordnung-zu-bekannten-Orten-in-Tabelle-place)  
[Orte in `place` eintragen](#Orte-in-place-eintragen)  
[Geburtsorte zuordnen](#Geburtsorte-zuordnen)  
[Verweise auf WHG eintragen](#Verweise-auf-WHG-eintragen)  

In [1]:
ENV["COLUMNS"] = 120

120

In [2]:
wds_path = "C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

"C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

In [3]:
using Pkg

In [4]:
Pkg.activate(wds_path)

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using JSON, DataFrames, CSV

In [6]:
using WiagDataSetup

In [7]:
Wds = WiagDataSetup

WiagDataSetup

In [8]:
computer = :bkroege_goe
computer = :ghertko_kn
if computer == :bkroege_goe
    data_path = "../data"
    out_path = "../data_sql"
else
    data_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"
    out_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"
end

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"

In [9]:
file_name = joinpath(data_path, "AnonymousUser_ordinations_788_20220329_145955.json")

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data\\AnonymousUser_ordinations_788_20220329_145955.json"

In [10]:
d_place = JSON.parsefile(file_name);

In [11]:
keys(d_place)

KeySet for a Dict{String, Any} with 4 entries. Keys:
  "@context"
  "features"
  "filename"
  "type"

In [12]:
features = d_place["features"];

In [13]:
length(features)

788

In [14]:
features[25]["@id"]

"brom_1052"

In [15]:
features[25]["links"][1]["type"], features[25]["links"][1]["identifier"]

("closeMatch", "wd:Q18982898")

In [15]:
get_whg_id(a) = replace(a, "http://whgazetteer.org/api/place/" => "")

get_whg_id (generic function with 1 method)

In [16]:
make_row(df::AbstractDataFrame) = Dict{Symbol, Any}(a => missing for a in Symbol.(names(df)))

make_row (generic function with 1 method)

Schreibe einen Eintrag für jede Namensvariante

In [198]:
df_place = DataFrame(
    id = String[],
    whg_id = String[],
    title = String[],    
    link_type = Union{String, Missing}[],
    link_identifier = Union{String, Missing}[],
    type_label = Union{String, Missing}[],
    type_identifier = Union{String, Missing}[],
    longitude = Union{Float64, Missing}[],
    latitude = Union{Float64, Missing}[],
);

In [20]:
make_row(df_place)

Dict{Symbol, Any} with 9 entries:
  :coo_0           => missing
  :coo_1           => missing
  :link_identifier => missing
  :id              => missing
  :link_type       => missing
  :type_label      => missing
  :title           => missing
  :whg_id          => missing
  :type_identifier => missing

Befülle `df` mit den Daten aus `features` (aus JSON-File).

In [39]:
features[223]["links"]

6-element Vector{Any}:
 Dict{String, Any}("identifier" => "primaryTopicOf: wp:Oudewater (lungsod sa Olandres)", "type" => "closeMatch")
 Dict{String, Any}("identifier" => "tgn:1048037", "type" => "closeMatch")
 Dict{String, Any}("identifier" => "wd:Q33470616", "type" => "closeMatch")
 Dict{String, Any}("identifier" => "loc:n83052580", "type" => "closeMatch")
 Dict{String, Any}("identifier" => "gn:2748979", "type" => "closeMatch")
 Dict{String, Any}("identifier" => "viaf:125524894", "type" => "closeMatch")

Erzeuge einen Eintrag in `df` für jeden Link

In [197]:
function fill_df_link(df_link, features)
    i = 0
    n_max = Inf
    for f in features
        pr = make_row(df_link)
        pr[:id] = f["@id"]
        pr[:whg_id] = get_whg_id(f["properties"]["pid"])
        pr[:title] = f["properties"]["title"]
        for g in f["geometry"]["geometries"]
            if haskey(g, "citation")
                continue
            end
            if g["type"] == "Point"
                pr[:longitude] = g["coordinates"][1]
                pr[:latitude] = g["coordinates"][2]
            end
        end
        has_links = false
        for l in f["links"]
            has_links = true
            pr[:link_identifier] = l["identifier"]
            push!(df_link, NamedTuple(pr))
        end    
        if !has_links
            push!(df_link, NamedTuple(pr))
        end
        i += 1
        if i > n_max
            break
        end
    end
end    

fill_df_link (generic function with 1 method)

In [199]:
fill_df_link(df_place, features)

In [23]:
names(df_place)

9-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "coo_0"
 "coo_1"

In [24]:
file_name = joinpath(data_path, "priest_utrecht_places_a.csv")
CSV.write(file_name, df_place, delim=";")

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data\\priest_utrecht_places_a.csv"

In [25]:
size(df_place)

(1460, 9)

In [26]:
length(unique(df_place.id))

788

## Zuordnung zu bekannten Orten in Tabelle `place`
Die Koordinaten sind vermutlich in WGS84 angegeben, wie in `place` auch.

Die Zuordnung wird zunächst nicht verwendet. Die Orte werden unter einem eigenen Typ entsprechend ihrer Quelle in die Tabelle `place` aufgenommen.

In [ ]:
wds_path = "C:/Users/georg/Documents/projekte/WiagDataSetup.jl"

In [ ]:
cd(wds_path)

In [ ]:
using Pkg

In [ ]:
Pkg.activate(".")

In [ ]:
using WiagDataSetup

In [ ]:
Wds = WiagDataSetup

In [171]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [ ]:
using NearestNeighbors

In [ ]:
sql = "SELECT id, geonames_id, name, feature_code, latitude, longitude 
FROM place";
df_place_gn = Wds.sql_df(sql);

In [ ]:
p_data = Float64.(transpose(hcat(df_place_gn.longitude, df_place_gn.latitude)));

In [ ]:
tree = KDTree(p_data)

In [ ]:
df_place[50:55, [:id, :whg_id, :title, :coo_0, :coo_1]]

In [ ]:
p_sample = permutedims(hcat(df_place.coo_0, df_place.coo_1));

In [ ]:
range = 0.01
idx = inrange(tree, p_sample, range);

In [ ]:
size(idx)

In [ ]:
idx[100:105]

In [ ]:
count(x -> length(x) >= 2, idx), count(x -> length(x) == 1, idx), count(x -> length(x) == 0, idx)

In [ ]:
df_place[104, [:id, :whg_id, :title, :coo_0, :coo_1]]

In [ ]:
df_place_gn[[86491, 87462], :]

In [ ]:
df_place_gp = groupby(df_place, [:coo_0, :coo_1]);

Es gibt einen Ort bezogen auf die Koordinaten, der in der Quelle doppelt vorkommt.

In [ ]:
size(df_place_gp)

In [ ]:
for g in df_place_gp
    id = unique(g.id)
    if length(id) > 1
        println(id)
    end
end

Das scheint eine echte Dublette: Hasselt

Erstelle die Zuordnungstabelle `df_place_map`  
Füge `place_id` ein und verknüpfe mit `df_place_gn`

In [ ]:
df_place_map = similar(df_place, 0);

In [ ]:
df_place_map[!, :place_id] = Vector{Union{Int, Missing}}()
df_place_map[!, :n_match] = Vector{Int}()

In [ ]:
for (row, i_nn) in zip(eachrow(df_place), idx)
    row_data = collect(row)
    row_add = [missing, 0]
    n_match = length(i_nn)
    if n_match == 0
        push!(df_place_map, vcat(row_data, row_add))
    else
        for id in i_nn
            row_add = [df_place_gn[id, :id], n_match]
            push!(df_place_map, vcat(row_data, row_add))
        end
    end
end            

In [ ]:
size(df_place_map)

In [ ]:
df_place_map = leftjoin(df_place_map, df_place_gn, on = :place_id => :id, matchmissing = :notequal);

In [ ]:
size(df_place_map)

In [ ]:
sort!(df_place_map, :id);

In [ ]:
file_name = joinpath(data_path, "priest_utrecht_places_geonames.csv")
CSV.write(file_name, df_place_map, delim=";")

In [ ]:
df_place_map_gp = groupby(df_place_map, :n_match);

In [ ]:
typeof(df_place_map_gp[1])

In [ ]:
propertynames(df_place_map_gp)

In [ ]:
df_place_map_gp.keymap

In [ ]:
df_place_map_gp[3][7, [:name, :n_match]]

In [ ]:
combine(df_place_map_gp, :id => (x -> length(unique(x))) => :n_id)

In [ ]:
sum([90, 531, 137, 21, 6, 3])

## Orte in `place` eintragen
Quelle: `C:\Users\georg\Documents\projekte-doc\Utrecht-Priester\Ordinations of priests (identified place of origin).tsv`  
bzw. `data\AnonymousUser_ordinations_788_20220329_145955.json`

siehe oben

Variante zwei nur aus `data\AnonymousUser_ordinations_788_20220329_145955.json` siehe unten

In [ ]:
names(df_place)

In [ ]:
size(df_place)

Manche Einträge enthalten im Feld `link_identifier` einen Verweis auf eine GeoNames ID. Lies sie aus und schreibe sie in das Feld `geonames_id`.

In [ ]:
rgx = r"^gn:([0-9]+)"
function get_id_gn_maybe(s)
    if ismissing(s)
        return missing
    end
    rgm = match(rgx, s)
    if !isnothing(rgm)
        return(parse(Int,rgm[1]))
    else
        return missing
    end
end 

In [ ]:
transform!(df_place, :link_identifier => ByRow(get_id_gn_maybe) => :geonames_id);

In Bezug auf die folgenden Spalten ist jeweils nur ein Eintrag relevant.

In [196]:
columns = [
    :id => :id_in_source,
    :geonames_id => :geonames_id,
    :title => :name,    
    :longitude => :longitude,
    :latitude => :latitude,
];

In [ ]:
df_place_ut = unique(select(df_place, columns));

In [ ]:
size(df_place_ut)

In [ ]:
df_place_ut_gn = subset(df_place_ut, :geonames_id => ByRow(!ismissing));

In [ ]:
size(df_place_ut_gn, 1), length(unique(df_place_ut_gn.id_in_source)) 

Einträge mit mehreren Verweisen auf GeoNames

In [ ]:
df_place_ut_gn_by_id = groupby(df_place_ut_gn, :id_in_source);

In [ ]:
df_tmp_count = combine(df_place_ut_gn_by_id, nrow => :n);

In [ ]:
df_place_ut_gn_n = leftjoin(df_place_ut_gn, df_tmp_count, on = :id_in_source);

In [ ]:
subset(df_place_ut_gn_n, :n => ByRow((x) -> (x > 1)))

In [ ]:
df_place_ut_gn_mg = antijoin(df_place_ut, df_place_ut_gn, on = :id_in_source);

In [ ]:
length(unique(df_place_ut_gn_mg.id_in_source))

In [ ]:
df_place_ut_out = vcat(df_place_ut_gn, df_place_ut_gn_mg);

In [ ]:
insertcols!(df_place_ut_out, :place_type_id => 2);

In [ ]:
table_name = "place"
file_name = joinpath(out_path, "$(table_name)_ut.sql")
Wds.insert_sql(file_name, table_name, df_place_ut_out);

In [ ]:
insertcols!(df_place_ut_gn_mg, :place_type_id => 2);

In [ ]:
out_path

```sql
source "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql/place_ut.sql";
```

#### Variante zwei

identifiziere Verweise auf GeoNames

In [214]:
function get_gn_id(s)
    if !ismissing(s) && s[1:3] == "gn:"
        return s[4:end]
    else
        return missing
    end
end     

get_gn_id (generic function with 1 method)

In [216]:
transform!(df_place, :link_identifier => ByRow(get_gn_id) => :geonames_id);

In [217]:
df_place_ug = groupby(df_place, :id);

In [218]:
df_place_u = combine(df_place_ug, first);

In [219]:
names(df_place_u)

10-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "longitude"
 "latitude"
 "geonames_id"

In [220]:
size(df_place_u)

(788, 10)

In [221]:
count(!ismissing, df_place_u.geonames_id)

26

In der Quelle werden einem Ort bisweilen auch mehrere GeoName-IDs zugewiesen?!

In [222]:
insertcols!(df_place_u, :place_type_id => 2);

In [223]:
names(df_place_u)

11-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "longitude"
 "latitude"
 "geonames_id"
 "place_type_id"

In [225]:
columns = [
    :id => :id_in_source,
    :geonames_id => :geonames_id,
    :place_type_id => :place_type_id,
    :title => :name,
    :longitude => :longitude,
    :latitude => :latitude,
]

6-element Vector{Pair{Symbol, Symbol}}:
            :id => :id_in_source
   :geonames_id => :geonames_id
 :place_type_id => :place_type_id
         :title => :name
     :longitude => :longitude
      :latitude => :latitude

In [227]:
table_name = "place"
file_name = joinpath(out_path, "$(table_name)_ut.sql")
Wds.insert_sql(file_name, table_name, select(df_place_u, columns));

Spiele IDs zurück

In [229]:
sql = "SELECT id as place_id, id_in_source FROM place WHERE place_type_id = 2";
df_idx = Wds.sql_df(sql);

In [230]:
df_place_u = leftjoin(df_place_u, df_idx, on = :id => :id_in_source);

In [231]:
size(df_place_u)

(788, 12)

## Verweise auf WHG eintragen

In [232]:
df_place_u[51:55, :]

,id,whg_id,title,link_type,link_identifier,type_label,type_identifier,longitude,latitude,geonames_id
,String,String,String,String?,String?,String?,String?,Float64?,Float64?,String?
1,brom_1100,6666457,Noertwellen,missing,missing,missing,missing,3.7924,51.718,missing
2,brom_1102,6666458,Ellemaet,missing,missing,missing,missing,3.82961,51.7136,missing
3,brom_1103,6666459,Elkersee,missing,wd:Q2346411,missing,missing,3.84411,51.7252,missing
4,brom_1105,6666460,Brauvershaven,missing,wd:Q991009,missing,missing,3.9122,51.7254,missing
5,brom_1106,6666461,Nijkerck,missing,missing,missing,missing,3.89086,51.7006,missing


In [233]:
authority_id_whg = 54

54

In [234]:
names(df_place_u)

12-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "longitude"
 "latitude"
 "geonames_id"
 "place_type_id"
 "place_id"

In [235]:
columns = [
    :geonames_id => :geonames_id,
    :whg_id => :value,
    :place_id => :place_id
]

3-element Vector{Pair{Symbol, Symbol}}:
 :geonames_id => :geonames_id
      :whg_id => :value
    :place_id => :place_id

In [236]:
df_id_external = select(df_place_u, columns);

In [237]:
insertcols!(df_id_external, :authority_id => authority_id_whg);

In [238]:
count(ismissing, df_id_external.value)

0

In [239]:
table_name = "place_id_external"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_id_external);

## Ortsnamen eintragen

In [241]:
keys(features[17])

KeySet for a Dict{String, Any} with 11 entries. Keys:
  "types"
  "descriptions"
  "names"
  "depictions"
  "when"
  "geometry"
  "@id"
  "links"
  "properties"
  "relations"
  "type"

In [251]:
df_place_n = DataFrame(
    id = String[],
    whg_id = String[],
    title = String[],    
    name = String[],
    longitude = Union{Float64, Missing}[],
    latitude = Union{Float64, Missing}[],
);

In [244]:
function fill_df_name(df_name, features)
    i = 0
    n_max = Inf
    for f in features
        pr = make_row(df_name)
        pr[:id] = f["@id"]
        pr[:whg_id] = get_whg_id(f["properties"]["pid"])
        title = f["properties"]["title"]
        pr[:title] = title
        for g in f["geometry"]["geometries"]
            if haskey(g, "citation")
                continue
            end
            if g["type"] == "Point"
                pr[:longitude] = g["coordinates"][1]
                pr[:latitude] = g["coordinates"][2]
            end
        end 
        for name_group in f["names"]
            name = name_group["toponym"]
            if name == ""
                name = title
            end
            pr[:name] = name
            push!(df_name, NamedTuple(pr))
        end
        i += 1
        if i > n_max
            break
        end
    end
    return i
end    

fill_df_name (generic function with 1 method)

In [252]:
fill_df_name(df_place_n, features)

788

In [247]:
size(df_place_n)

(1985, 6)

IDs zuweisen

In [253]:
sql = "SELECT id as place_id, id_in_source, geonames_id FROM place WHERE place_type_id = 2";
df_idx = Wds.sql_df(sql);

In [254]:
df_place_n = leftjoin(df_place_n, df_idx, on = :id => :id_in_source);

In [249]:
size(df_place_n)

(1985, 7)

In [255]:
names(df_place_n)

8-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "name"
 "longitude"
 "latitude"
 "place_id"
 "geonames_id"

In [256]:
columns = [
    :place_id => :place_id,
    :geonames_id => :geonames_id,
    :name => :label
]

3-element Vector{Pair{Symbol, Symbol}}:
    :place_id => :place_id
 :geonames_id => :geonames_id
        :name => :label

In [264]:
df_label = select(df_place_n, columns);

In [265]:
insertcols!(df_label,
    :is_preferred => 0,
    :is_historic => 1,
    :is_geonames_name => 0,
);

In [266]:
table_name = "place_label"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_label);

## Geburtsorte zuordnen (erste Version)
Stapel hat für einen Teil der Geburtsorte Koordinaten angegeben. Sie werden genutzt für eine Zuordnung zu Orten in Tabelle `place`.  
Siehe zweite Version unten.

In [25]:
file_name = joinpath(data_path, "..", "Ordinations of priests (identified place of origin).tsv")
df_priest_bp_in = CSV.read(file_name, DataFrame, delim = "\t");

┌ Warning: thread = 2 warning: only found 25 / 26 columns around data row: 5480. Filling remaining columns with `missing`
└ @ CSV C:\Users\georg\.julia\packages\CSV\b4GfC\src\file.jl:622


In [26]:
size(df_priest_bp_in)

(5255, 26)

In [27]:
df_priest_bp_in[301:303, [:WKT_GEOM, :Priester_ID, :Toponiem, :Voornaam]]

,WKT_GEOM,Priester_ID,Toponiem,Voornaam
,String,String15,String31?,String15
1,Point (6.0937644735630796 52.51259100584277917),id_1513_4_55,Zvollis,Hermannus
2,Point (6.0937644735630796 52.51259100584277917),id_1513_4_57,Zvollis,Bernardus
3,Point (6.0937644735630796 52.51259100584277917),id_1513_6_12,Zvollis,Lubbertus


In [28]:
rgx = r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"

r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"

In [29]:
rgm = match(rgx, "Point (6.0937644735630796 52.51259100584277917)")

RegexMatch("Point (6.0937644735630796 52.51259100584277917)", 1="6.0937644735630796", 2="52.51259100584277917")

In [30]:
parse(Float64, rgm[1])

6.09376447356308

In [31]:
function parse_wkt_point(wkt)
    rgx = r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"
    longitude = missing
    latitude = missing
    rgm = match(rgx, wkt)
    if !isnothing(rgm)
        if !isnothing(rgm[1])
            longitude = parse(Float64, rgm[1])
        end
        if !isnothing(rgm[2])
            latitude = parse(Float64, rgm[2])
        end
    end
    return longitude, latitude
end    

parse_wkt_point (generic function with 1 method)

In [32]:
names(df_priest_bp_in)

26-element Vector{String}:
 "WKT_GEOM"
 "FACTOR"
 "Bron"
 "Priester_ID"
 "Datum"
 "Locatie"
 "Transcriptie"
 "Voetnoot_bron"
 "Bijdrage"
 "Titel"
 "Voornaam"
 "Patroniem"
 "Familienaam"
 "Familienaam, toponiem"
 "Familienaam, onzeker"
 "Toponiem"
 "Orde"
 "Functie"
 "Instelling"
 "Voornaam_norm"
 "Patroniem_norm"
 "Orde_norm"
 "Opmerkingen"
 "Voornaam_uniciteit"
 "Patroniem_uniciteit"
 "Naam_ARDOU"

In [33]:
columns = [
    :WKT_GEOM => :WKT_GEOM,
    :Priester_ID => :priest_id_in_source,
    :Voornaam => :givenname,
    :Patroniem => :patronym,
    :Toponiem => :place_name_in,
    :FACTOR => :place_weight,
]

6-element Vector{Pair{Symbol, Symbol}}:
    :WKT_GEOM => :WKT_GEOM
 :Priester_ID => :priest_id_in_source
    :Voornaam => :givenname
   :Patroniem => :patronym
    :Toponiem => :place_name_in
      :FACTOR => :place_weight

In [34]:
df_priest_bp = select(df_priest_bp_in, columns);

In [35]:
transform!(df_priest_bp, :WKT_GEOM => ByRow(parse_wkt_point) => [:longitude, :latitude]);

In [36]:
size(df_priest_bp), length(unique(df_priest_bp.priest_id_in_source))

((5255, 8), 3958)

In [38]:
df_priest_bp_n = groupby(df_priest_bp, :priest_id_in_source)

GroupedDataFrame with 3958 groups based on key: priest_id_in_source
First Group (2 rows): priest_id_in_source = "id_1516_2_5"
 Row │ WKT_GEOM                           priest_id_in_source  givenname  patronym   place_name_in  place_weight  long ⋯
     │ String                             String15             String15   String31?  String31?      Float64?      Floa ⋯
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Point (6.15866341162453335 53.47…  id_1516_2_5          Johannes   Nicolai    Oculo              0.629069    6. ⋯
   2 │ Point (4.69987561257871622 52.83…  id_1516_2_5          Johannes   Nicolai    Oculo              0.370931    4.
                                                                                                       2 columns omitted
⋮
Last Group (1 row): priest_id_in_source = "id_1509_3_93"
 Row │ WKT_GEOM                           priest_id_in_source  givenname  patronym   place_name_in  place_weight  long ⋯
     │ String                             String15             String15   String31?  String31?      Float64?      Floa ⋯
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Point (6.008826 52.5206140000000…  id_1509_3_93         Gerardus   Lamberti   Sallick                 1.0    6. ⋯
                                                                                                       2 columns omitted

In [39]:
df_priest_bp_count = combine(df_priest_bp_n, nrow);

In [40]:
df_priest_bp_mult = subset(df_priest_bp_count, :nrow => ByRow((x) -> (x > 1)));

In [41]:
df_priest_bp_mult[51:55, :]

,priest_id_in_source,nrow
,String15,Int64
1,id_1516_3_104,6
2,id_1516_6_12,6
3,id_1517_4_35,6
4,id_1517_4_54,6
5,id_1516_4_29,2


*2022-04-12*: Es ist nicht klar, wie die Zuordnung der Koordinaten zu interpretieren ist. Bei mehrfachen Zuordnungen gibt es einen Faktor. Die Summe der Faktoren ist 1.0.  
*2022-04-14*: Interpretiere den Faktor als eine Gewichtung für eine zutreffende Zuordnung.

In [42]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [43]:
using NearestNeighbors

In [44]:
sql = "SELECT id, geonames_id, name, feature_code, latitude, longitude, place_type_id 
FROM place";
df_place_gn = Wds.sql_df(sql);

In [45]:
transform!(df_place_gn, :longitude => ByRow(Float64) => :longitude);
transform!(df_place_gn, :latitude => ByRow(Float64) => :latitude);

In [46]:
p_data = Float64.(permutedims(hcat(df_place_gn.longitude, df_place_gn.latitude)));

In [47]:
tree = KDTree(p_data)

KDTree{StaticArrays.SVector{2, Float64}, Euclidean, Float64}
  Number of points: 392036
  Dimensions: 2
  Metric: Euclidean(0.0)
  Reordered: true

In [48]:
names(df_priest_bp)

8-element Vector{String}:
 "WKT_GEOM"
 "priest_id_in_source"
 "givenname"
 "patronym"
 "place_name_in"
 "place_weight"
 "longitude"
 "latitude"

In [49]:
p_sample = permutedims(hcat(df_priest_bp.longitude, df_priest_bp.latitude));

In [50]:
range = 0.007
idx = inrange(tree, p_sample, range);

In [51]:
size(idx)

(5255,)

In [43]:
idx[100:105]

6-element Vector{Vector{Int64}}:
 [391796, 84994, 82162]
 [391796, 84994, 82162]
 [391796, 84994, 82162]
 [87058, 391797]
 [87058, 391797]
 [87058, 391797]

In [52]:
count(x -> length(x) >= 2, idx), count(x -> length(x) == 1, idx), count(x -> length(x) == 0, idx)

(3605, 1510, 140)

Beschränke die Suche auf den nächsten Nachbarn

In [53]:
idx, dist = nn(tree, p_sample);

In [54]:
insertcols!(df_priest_bp,
    :idx => idx,
    :dist => dist);

In [55]:
get_place_gn(idx) = df_place_gn[idx, :id], df_place_gn

get_place_gn (generic function with 1 method)

In [56]:
get_place_gn(idx) = Tuple(df_place_gn[idx, col] for col in (:id, :place_type_id, :name))

get_place_gn (generic function with 1 method)

In [57]:
th = 0.007
get_place_gn_maybe(idx, dist) = dist < 0.007 ? get_place_gn(idx) : (missing, missing, missing)

get_place_gn_maybe (generic function with 1 method)

In [58]:
get_place_gn_maybe(300, 0.1e-3)

(350633, 1, "Zellerreit")

In [59]:
df_priest_bp_place = transform(df_priest_bp,
[:idx, :dist] => ByRow(get_place_gn_maybe) => [:place_id, :place_type_id, :place_name]);

In [60]:
columns = [
    :priest_id_in_source,
    :givenname,
    :place_id,
    :place_type_id,
    :place_name_in,
    :place_name
]

6-element Vector{Symbol}:
 :priest_id_in_source
 :givenname
 :place_id
 :place_type_id
 :place_name_in
 :place_name

In [61]:
df_priest_bp_place[100:107, columns]

,priest_id_in_source,givenname,place_id,place_type_id,place_name_in,place_name
,String15,String15,Int32?,Int32?,String31?,String?
1,id_1505_2_44,Heinricus,944237,2,Dieveren,Dieveren
2,id_1512_3_54,Helpricus,944237,2,Dieveren,Dieveren
3,id_1517_4_4,Coenrardus,944237,2,Dieveren,Dieveren
4,id_1505_2_44,Heinricus,944238,2,Dieveren,Dieveren
5,id_1512_3_54,Helpricus,944238,2,Dieveren,Dieveren
6,id_1517_4_4,Coenrardus,944238,2,Dieveren,Dieveren
7,id_1517_3_49,Johannes,944239,2,Wapservenis,Wapservenis
8,id_1511_4_32,Theodricus,944240,2,Dolre,Dolre


In [62]:
count(ismissing, df_priest_bp_place.place_weight)

71

In [63]:
names(df_priest_bp_place)

13-element Vector{String}:
 "WKT_GEOM"
 "priest_id_in_source"
 "givenname"
 "patronym"
 "place_name_in"
 "place_weight"
 "longitude"
 "latitude"
 "idx"
 "dist"
 "place_id"
 "place_type_id"
 "place_name"

In [64]:
priest_type_id = 10
sql = "SELECT id, id_in_source 
FROM item
WHERE item_type_id = $(priest_type_id)";
df_item = Wds.sql_df(sql);

In [65]:
columns = [
    :priest_id_in_source => :id_in_source,
    :place_name_in => :place_name,
    :place_weight => :weight,
    :place_id => :place_id
]

4-element Vector{Pair{Symbol, Symbol}}:
 :priest_id_in_source => :id_in_source
       :place_name_in => :place_name
        :place_weight => :weight
            :place_id => :place_id

In [66]:
df_priest_bp_out = leftjoin(select(df_priest_bp_place, columns), df_item, on = :id_in_source);

In [67]:
count(ismissing, df_priest_bp_out.id)

0

In [68]:
names(df_priest_bp_out)

5-element Vector{String}:
 "id_in_source"
 "place_name"
 "weight"
 "place_id"
 "id"

In [69]:
dropmissing!(df_priest_bp_out, :weight);

In [70]:
round6(d) = round(d, digits = 6)
transform!(df_priest_bp_out, :weight => ByRow(round6) => :weight6);

In [71]:
size(df_priest_bp_out)

(5184, 6)

In [72]:
df_priest_bp_out[400:407, :]

,id_in_source,place_name,weight,place_id,id,weight6
,String15,String31?,Float64,Int32?,Int32?,Float64
1,id_1514_4_17,Campis,0.631456,944365,315383,0.631456
2,id_1514_5_6,Cempis,0.631456,944365,315464,0.631456
3,id_1514_5_18,Campis,0.631456,944365,315476,0.631456
4,id_1514_5_32,Campis,0.631456,944365,315490,0.631456
5,id_1514_5_38,Kemponis,1.0,944365,315496,1.0
6,id_1514_6_4,Campis,0.631456,944365,315507,0.631456
7,id_1515_1_14,Campis,0.631456,944365,315541,0.631456
8,id_1515_4_59,Campis,0.631456,944365,315758,0.631456


In [73]:
columns = [
    :id => :person_id,
    :place_id => :place_id,
    :place_name => :place_name,
    :weight6 => :weight
]

4-element Vector{Pair{Symbol, Symbol}}:
         :id => :person_id
   :place_id => :place_id
 :place_name => :place_name
    :weight6 => :weight

In [74]:
table_name = "person_birthplace";
file_name = joinpath(out_path, "$(table_name).sql");
Wds.insert_sql(file_name, table_name, select(df_priest_bp_out, columns))

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


5184

In [184]:
select(df_priest_bp_out, columns)[400:407, :]

,person_id,place_id,place_name,weight
,Int32?,Int32?,String31?,Float64
1,180500,944365,Campis,0.631456
2,180501,944365,Cempis,0.631456
3,180502,944365,Campis,0.631456
4,180503,944365,Campis,0.631456
5,180504,944365,Kemponis,1.0
6,180505,944365,Campis,0.631456
7,180506,944365,Campis,0.631456
8,180507,944365,Campis,0.631456


## Geburtsorte zuordnen

In [267]:
names(df_place_u)

12-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "longitude"
 "latitude"
 "geonames_id"
 "place_type_id"
 "place_id"

Tabelle mit Koordinaten, Ortsnamen und Priestern

In [268]:
file_name = joinpath(data_path, "..", "Ordinations of priests (identified place of origin).tsv")
df_bp_in = CSV.read(file_name, DataFrame, delim = "\t", stringtype = String);

┌ Warning: thread = 2 warning: only found 25 / 26 columns around data row: 5480. Filling remaining columns with `missing`
└ @ CSV C:\Users\georg\.julia\packages\CSV\b4GfC\src\file.jl:622


In [269]:
size(df_bp_in)

(5255, 26)

In [270]:
df_bp_in[301:303, [:WKT_GEOM, :Priester_ID, :Toponiem, :Voornaam]]

,WKT_GEOM,Priester_ID,Toponiem,Voornaam
,String,String,String?,String
1,Point (6.0937644735630796 52.51259100584277917),id_1513_4_55,Zvollis,Hermannus
2,Point (6.0937644735630796 52.51259100584277917),id_1513_4_57,Zvollis,Bernardus
3,Point (6.0937644735630796 52.51259100584277917),id_1513_6_12,Zvollis,Lubbertus


In [151]:
rgx = r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"

r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"

In [152]:
rgm = match(rgx, "Point (6.0937644735630796 52.51259100584277917)")

RegexMatch("Point (6.0937644735630796 52.51259100584277917)", 1="6.0937644735630796", 2="52.51259100584277917")

In [153]:
parse(Float64, rgm[1])

6.09376447356308

In [271]:
function parse_wkt_point(wkt)
    rgx = r"Point +\(([0-9]+\.[0-9]+) +([0-9]+\.[0-9]+)\)"
    longitude = missing
    latitude = missing
    rgm = match(rgx, wkt)
    if !isnothing(rgm)
        if !isnothing(rgm[1])
            longitude = parse(Float64, rgm[1])
        end
        if !isnothing(rgm[2])
            latitude = parse(Float64, rgm[2])
        end
    end
    return longitude, latitude
end    

parse_wkt_point (generic function with 1 method)

In [272]:
columns = [
    :WKT_GEOM => :WKT_GEOM,
    :Priester_ID => :priest_id_in_source,
    :Voornaam => :givenname,
    :Patroniem => :patronym,
    :Toponiem => :place_name_in,
    :FACTOR => :place_weight,
]

6-element Vector{Pair{Symbol, Symbol}}:
    :WKT_GEOM => :WKT_GEOM
 :Priester_ID => :priest_id_in_source
    :Voornaam => :givenname
   :Patroniem => :patronym
    :Toponiem => :place_name_in
      :FACTOR => :place_weight

In [273]:
df_bp = select(df_bp_in, columns);

In [274]:
transform!(df_bp, :WKT_GEOM => ByRow(parse_wkt_point) => [:longitude_bp, :latitude_bp]);

In [275]:
size(df_bp), length(unique(df_bp.priest_id_in_source))

((5255, 8), 3958)

Bei Einträgen ohne Ortsnamen fehlt auch immer ein Eintrag in der Spalte `Factor`.

In [96]:
delta_geo(e1, n1, e2, n2) = ismissing(e1 + n1 + e2 + n2) ? 10 : sqrt(abs2(e2 - e1) + abs(n2 - n1))

delta_geo (generic function with 1 method)

Finde einen passenden Ort

In [289]:
df_bp_whg = copy(df_bp);

In [290]:
s1 = size(df_priest_bp_whg, 1)

5255

In [292]:
insertcols!(df_bp_whg,
    :brom_id => Array{Union{Missing, String}}(missing, s1),
    :whg_id => Array{Union{Missing, String}}(missing, s1),
    :name_whg => Array{Union{Missing, String}}(missing, s1),
    :place_id => Array{Union{Missing, Int}}(missing, s1)
);

In [293]:
delta_geo(e1, n1, e2, n2) = ismissing(e1 + n1 + e2 + n2) ? 10 : sqrt(abs2(e2 - e1) + abs(n2 - n1))

delta_geo (generic function with 1 method)

In [297]:
function find_bp(df_priest, df_place, th = 0.001)
    n_mg = 0
    for row_p in eachrow(df_priest)
        flag_found = false
        for row_cand in eachrow(df_place)
            delta = delta_geo(row_p.longitude_bp, row_p.latitude_bp, row_cand.longitude, row_cand.latitude)
            if delta < th
                row_p.brom_id = row_cand.id
                row_p.whg_id = row_cand.whg_id
                row_p.name_whg = row_cand.title
                row_p.place_id = row_cand.place_id
                flag_found = true
                break
            end
        end
        flag_found ? continue : n_mg += 1
    end
    return n_mg
end

find_bp (generic function with 2 methods)

In [295]:
names(df_place_u)

12-element Vector{String}:
 "id"
 "whg_id"
 "title"
 "link_type"
 "link_identifier"
 "type_label"
 "type_identifier"
 "longitude"
 "latitude"
 "geonames_id"
 "place_type_id"
 "place_id"

In [298]:
find_bp(df_bp_whg, df_place_u)

226

begin info

In [299]:
df_new_name = subset(df_bp_whg, [:place_name_in, :name_whg] => ByRow(!isequal));

In [300]:
size(df_new_name)

(3270, 12)

In [301]:
df_new_name[301:305, [:priest_id_in_source, :brom_id, :place_name_in, :name_whg]]

,priest_id_in_source,brom_id,place_name_in,name_whg
,String,String?,String?,String?
1,id_1513_4_44,brom_608,Ghoer,Goer
2,id_1514_1_12,brom_608,Ghoer,Goer
3,id_1514_3_36,brom_608,Ghoer,Goer
4,id_1516_3_101,brom_608,Ghoer,Goer
5,id_1517_3_57,brom_608,Ghoer,Goer


end info

Verknüpfe `item.id` als `person_id`

In [302]:
sql = "SELECT id as person_id, id_in_source FROM item WHERE item_type_id = 10";
df_idx = Wds.sql_df(sql);

In [304]:
df_bp_whg = leftjoin(df_bp_whg, df_idx, on = :priest_id_in_source => :id_in_source);

In [305]:
size(df_bp_whg)

(5255, 13)

In [311]:
count(ismissing, df_bp_whg.person_id), count(ismissing, df_bp_whg.place_name_in)

(0, 64)

In [313]:
dropmissing!(df_bp_whg, :place_name_in);

In [307]:
names(df_bp_whg)

13-element Vector{String}:
 "WKT_GEOM"
 "priest_id_in_source"
 "givenname"
 "patronym"
 "place_name_in"
 "place_weight"
 "longitude_bp"
 "latitude_bp"
 "brom_id"
 "whg_id"
 "name_whg"
 "place_id"
 "person_id"

In [314]:
columns = [
    :person_id => :person_id,
    :place_name_in => :place_name,
    :place_id => :place_id,
    :place_weight => :weight,
]

4-element Vector{Pair{Symbol, Symbol}}:
     :person_id => :person_id
 :place_name_in => :place_name
      :place_id => :place_id
  :place_weight => :weight

In [315]:
table_name = "person_birthplace";
file_name = joinpath(out_path, "$(table_name).sql");
Wds.insert_sql(file_name, table_name, select(df_bp_whg, columns))

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


5191